In [1]:
'''
差分内容
- week dayofyer
0.36472
- storeテーブルのPromo2SinceWeek	Promo2SinceYear	PromoIntervalの情報以外を使用してtrainにMERGE

'''

'\n差分内容\n\nベースライン作成\n時系列クロスバリデーション\n\n複数model保存・読み出し・予測→重み付き平均\n重み付き平均はデータ量が多いほど重みを付けたがもっと適切なものを探る\n\nPB score\n0.56260\n'

In [124]:
# main module
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import seaborn as sns

import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error

import datetime
import gc
import logging
import pickle
from pickle import load
import sys, os

# -------------------------------------独自モジュール-------------------------------------
sys.path.append('../src/') #モジュールが入っているディレクトリのパスを指定
import eda
import maprepro as mpre
import maprepro2 as mpre2
# import config
# from utils import setup_logger, ModelFactory
# -------------------------------------独自モジュール-------------------------------------

In [150]:
path = '../../input'
sample = pd.read_csv(f'{path}/sample_submission.csv')
store = pd.read_csv(f'{path}/store.csv')

test = pd.read_csv(f'{path}/test.csv',
                       parse_dates=['Date'],
                       date_parser=(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d')))

train = pd.read_csv(f'{path}/train.csv',
                       parse_dates=['Date'],
                       date_parser=(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d')))


In [151]:
def mk_cat(df):
    # intなので決定木系でないと使えない処理
    store['StoreType'] = store['StoreType'].astype('category').cat.codes
    store['Assortment'] = store['Assortment'].astype('category').cat.codes
    return store
store = mk_cat(store)
use = ['Store','StoreType','Assortment','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2']
train = pd.merge(train, store[use], how='left', on=['Store'])
test = pd.merge(test, store[use], how='left', on=['Store'])

In [152]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2
0,1,5,2015-07-31,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,0,0,1900.0,6.0,2014.0,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,2,2,1880.0,4.0,2006.0,0
1017206,1113,2,2013-01-01,0,0,0,0,a,1,0,2,9260.0,NaN,NaN,0
1017207,1114,2,2013-01-01,0,0,0,0,a,1,0,2,870.0,NaN,NaN,0


In [153]:
def mk_YMD_DOY(df):
    var_name = 'Date'
    df[var_name + 'Year'] = pd.Index(df[var_name]).year
    df[var_name + 'Month'] = pd.Index(df[var_name]).month
    df[var_name + 'Week'] = pd.Index(df[var_name]).week
    df[var_name + 'Day'] = pd.Index(df[var_name]).day
    df[var_name + 'DayOfYear'] = pd.Index(df[var_name]).dayofyear
    return df
train = mk_YMD_DOY(train)
test = mk_YMD_DOY(test)

# 時系列データであり、時間に沿って変数periodを設定したとする
def mk_period(df,testdata=False):
    if testdata == False:
        df['period'] = np.arange(0, len(df)) // (len(df) // 4)
        df['period'] = np.clip(df['period'], 0, 3)
        return df
    else:
        df['period'] = 4
        return df

train = mk_period(train)
test = mk_period(test,testdata=True)

In [154]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,DateYear,DateMonth,DateWeek,DateDay,DateDayOfYear,period
0,1,5,2015-07-31,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,2015,7,31,31,212,0
1,2,5,2015-07-31,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,2015,7,31,31,212,0
2,3,5,2015-07-31,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1,2015,7,31,31,212,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0,2015,7,31,31,212,0
4,5,5,2015-07-31,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0,2015,7,31,31,212,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,0,0,1900.0,6.0,2014.0,1,2013,1,1,1,1,3
1017205,1112,2,2013-01-01,0,0,0,0,a,1,2,2,1880.0,4.0,2006.0,0,2013,1,1,1,1,3
1017206,1113,2,2013-01-01,0,0,0,0,a,1,0,2,9260.0,NaN,NaN,0,2013,1,1,1,1,3
1017207,1114,2,2013-01-01,0,0,0,0,a,1,0,2,870.0,NaN,NaN,0,2013,1,1,1,1,3


In [155]:
train['StateHoliday'] = train.StateHoliday.astype('category')
test['StateHoliday'] = test.StateHoliday.astype('category')

alltmp = sorted(train.columns)
target = ['Sales']
notuse = ['Id','Date']
use = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','period',
        'DateYear','DateMonth','DateWeek','DateDay']

# unknown--'Customers'column
use2 = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance',
        'CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2','DateYear','DateMonth','DateWeek','DateDay','DateDayOfYear','period']
        
categorical_features = ['StateHoliday']

train_y = train[target]
train_x = train[use2]
test_x = test[use2]

In [156]:
import warnings
warnings.simplefilter('ignore')
train_x = eda.reduce_mem_usage(train_x)

start size(BEFORE): 123.20 Mb
Mem. usage decreased to 32.98 Mb (AFTER:73.2% reduction)


In [159]:
# --------------------------------------------------
import maprepro2 as mpre2
NOW,TMP_DIR = mpre2.mk_dir()
gc.collect()
# --------------------------------------------------

va_period_list = [1, 2, 3]
for va_period in va_period_list:
    print('i ================================================================== ',va_period)
    is_tr = train_x['period'] < va_period
    is_va = train_x['period'] == va_period
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]
    
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータの設定
    params = {'objective': 'regression',
                'seed': 71,
                'verbose': 1,
                'metrics': 'rmse',
                'force_col_wise':'true' # メモリが足りないから
                }
    num_round = 100

    # 学習の実行
    categorical_features = categorical_features
    model = lgb.train(params, lgb_train, num_boost_round=num_round,
                    categorical_feature=categorical_features,
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval],
                    )


    tmpfile = f'{TMP_DIR}/trained_model{va_period}.pkl'
    pickle.dump(model, open(tmpfile, 'wb'))

    # バリデーションデータでのスコアの確認
    va_pred = model.predict(va_x); va_pred = va_pred.reshape(-1, 1)

    # Open=0の日はすべてSalesが0で、0以外を予測することはありえないので0で更新(モデルは自然数を予測しているみたいだが、
    # Open=0ならSales=0をモデルに教えられていないということ)
    tmp =va_y['Sales']==0
    va_pred[tmp]=0
    va_pred=va_pred+1; va_y=va_y+1

    # --------------------------------------------------
    # score = mean_squared_error(va_y, va_pred)
    RMSPE = np.sqrt(np.mean(((  (va_y-va_pred)/va_y)**2) )).values
    RMSPE = RMSPE.astype(float)[0] # printのために単一の数値にしたかった
    # score = log_loss(va_y, va_pred)
    print('RMSPE RMSPE RMSPE RMSPE RMSPE RMSPE: {:.4f}'.format(RMSPE))
    # --------------------------------------------------

    del model # 学習済みモデルを削除


i ==================================================================  1
[LightGBM] [Info] Total Bins 888
[LightGBM] [Info] Number of data points in the train set: 254302, number of used features: 17
[LightGBM] [Info] Start training from score 5933.110593
[1]	train's rmse: 3727.71	valid's rmse: 3571.03
[2]	train's rmse: 3512.91	valid's rmse: 3330.4
[3]	train's rmse: 3330.05	valid's rmse: 3145.04
[4]	train's rmse: 3173.24	valid's rmse: 2996.52
[5]	train's rmse: 3037.72	valid's rmse: 2884.9
[6]	train's rmse: 2920.33	valid's rmse: 2800.63
[7]	train's rmse: 2820.73	valid's rmse: 2730.21
[8]	train's rmse: 2735.86	valid's rmse: 2672.71
[9]	train's rmse: 2663.7	valid's rmse: 2623.73
[10]	train's rmse: 2601.94	valid's rmse: 2587.19
[11]	train's rmse: 2544.88	valid's rmse: 2549.35
[12]	train's rmse: 2501.37	valid's rmse: 2553.06
[13]	train's rmse: 2456.9	valid's rmse: 2528.3
[14]	train's rmse: 2421.53	valid's rmse: 2534.18
[15]	train's rmse: 2385.28	valid's rmse: 2512.6
[16]	train's rmse: 2360.0

In [160]:
modeldict = mpre2.load_models(TMP_DIR)

load開始
load終わりました


In [161]:
pred_dict = mpre2.mk_pred_dict(modeldict,test_x)

予測リストが入った辞書作成開始
予測リストが入った辞書作成終了


In [162]:
pred_wei_average = mpre2.wei_average(pred_dict,len(test_x))

複数モデルを使用した重み付き平均予測を開始します
重み付き平均pred_wei_averageがreturnされます
重み付き平均pred_wei_averageがreturnされました


In [163]:
subpred = pd.DataFrame(pred_wei_average)
# subpred.sort_values('Sales')

sub = pd.concat([test['Id'],subpred],axis=1)
sub.columns = ['Id','Sales']

In [164]:
mpre2.mk_output(df=sub,NOW=NOW,PRACTICE=False)